In [13]:
#some not used. will delete later
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

from string import punctuation
from sklearn import svm
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
from nltk import ngrams
from itertools import chain
from wordcloud import WordCloud

In [14]:
#read data frame
odf = pd.read_csv('/Users/Gianina/Documents/MSc/spark/notebooks/data.csv', encoding = 'unicode_escape')
odf.head()

,X,book_title,review_title,review_user,book_id,review_id,timestamp,review_text,review_score
0,1,A Gentleman in Moscow: A Novel,Russian aristocracy following the Russian Revo...,Kansabelle,0143110438,R2UFCQ9WES7VFH,1555241537,"A great read. In addition to the plot, charact...",4
1,2,A Gentleman in Moscow: A Novel,Knowing nothing ahead of time just walk into t...,D.P. McHenry,0143110438,R24B1HA9J9I99G,1555241542,"Great story, well told. Characters were well d...",4
2,3,Pet Sematary: A Novel,"One of King's finest, most frightening, & earl...",Gordon Hoffman,198211598X,R1P137WFADSBYR,1555241649,Only the second novel written and published un...,4
3,4,Less (Winner of the Pulitzer Prize): A Novel,Not my favorite,R. Zocher,0316316121,R35533AKR5CBNS,1555242044,This book is t what I expected. Story is a lit...,4
4,5,Supermarket,"AMAZING, BOBBY",D. Mahoney,1982127139,R1D6LXSDR0CRMN,1555242169,As someone who hates reading and never really ...,4


In [16]:
#merge review title and text into a single column 
odf['review_title_text'] = odf.agg('{0[review_title]}  {0[review_text]}'.format, axis=1)

#odf['list']=odf[['review_title', 'review_text']].values.tolist()
#odf['review_title+text']=odf['list'].apply(''.join)
print(odf)

         X                                         book_title  \
0        1                     A Gentleman in Moscow: A Novel   
1        2                     A Gentleman in Moscow: A Novel   
2        3                              Pet Sematary: A Novel   
3        4       Less (Winner of the Pulitzer Prize): A Novel   
4        5                                        Supermarket   
5        6                   The Woman in the Window: A Novel   
6        7                     A Gentleman in Moscow: A Novel   
7        8                   The Woman in the Window: A Novel   
8        9                                           Run Away   
9       10                     A Gentleman in Moscow: A Novel   
10      11                Metropolis (A Bernie Gunther Novel)   
11      12                   The Woman in the Window: A Novel   
12      13                                        Supermarket   
13      14                              Pet Sematary: A Novel   
14      15        QAnon: 

In [17]:
#remove neutral reviews (ranked 3)
df = odf[odf['review_score'] != 3]
X = df['review_title_text']
y_dict = {1:0, 2:0, 4:1, 5:1}
y = df['review_score'].map(y_dict)
df.head()
#X.head()
#y.head()

,X,book_title,review_title,review_user,book_id,review_id,timestamp,review_text,review_score,review_title_text
0,1,A Gentleman in Moscow: A Novel,Russian aristocracy following the Russian Revo...,Kansabelle,0143110438,R2UFCQ9WES7VFH,1555241537,"A great read. In addition to the plot, charact...",4,Russian aristocracy following the Russian Revo...
1,2,A Gentleman in Moscow: A Novel,Knowing nothing ahead of time just walk into t...,D.P. McHenry,0143110438,R24B1HA9J9I99G,1555241542,"Great story, well told. Characters were well d...",4,Knowing nothing ahead of time just walk into t...
2,3,Pet Sematary: A Novel,"One of King's finest, most frightening, & earl...",Gordon Hoffman,198211598X,R1P137WFADSBYR,1555241649,Only the second novel written and published un...,4,"One of King's finest, most frightening, & earl..."
3,4,Less (Winner of the Pulitzer Prize): A Novel,Not my favorite,R. Zocher,0316316121,R35533AKR5CBNS,1555242044,This book is t what I expected. Story is a lit...,4,Not my favorite This book is t what I expecte...
4,5,Supermarket,"AMAZING, BOBBY",D. Mahoney,1982127139,R1D6LXSDR0CRMN,1555242169,As someone who hates reading and never really ...,4,"AMAZING, BOBBY As someone who hates reading a..."


In [18]:
#logistic regression model on word count
c = CountVectorizer(stop_words = ('the', 'in', 'read', 'pages', 'his', 'was', 'what','would', 'am', 'so', 'is', 'from', 'that', 'will', 'then', 'up', 'still', 'did', 'also', 'through', 'about', 'this', 'you', 'and', 'or', 'for', 'at', 'as', 'to','of', 'a', 'an', 'but', 'when', 'since', 'because'))
tfidf = TfidfVectorizer(stop_words = 'english')

def text_fit(X, y, model,clf_model,coef_show=1):
    
    X_c = model.fit_transform(X)
    print('# features: {}'.format(X_c.shape[1]))
    X_train, X_test, y_train, y_test = train_test_split(X_c, y, random_state=0)
    print('# train records: {}'.format(X_train.shape[0]))
    print('# test records: {}'.format(X_test.shape[0]))
    clf = clf_model.fit(X_train, y_train)
    acc = clf.score(X_test, y_test)
    print ('Model Accuracy: {}'.format(acc))
    
    if coef_show == 1: 
        w = model.get_feature_names()
        coef = clf.coef_.tolist()[0]
        coeff_df = pd.DataFrame({'Word' : w, 'Coefficient' : coef})
        coeff_df = coeff_df.sort_values(['Coefficient', 'Word'], ascending=[0, 1])
        print('')
        print('-Top 20 positive-')
        print(coeff_df.head(20).to_string(index=False))
        print('')
        print('-Top 20 negative-')        
        print(coeff_df.tail(20).to_string(index=False))
    
    
text_fit(X, y, c, LogisticRegression())

# features: 15728
# train records: 3039
# test records: 1013


C:\Users\Gianina\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Model Accuracy: 0.9516288252714709

-Top 20 positive-
       Word  Coefficient
       five     1.393591
      great     0.995946
  excellent     0.925243
        put     0.862109
       well     0.843585
  recommend     0.815021
      years     0.785242
  wonderful     0.782149
      loved     0.766732
      hotel     0.760652
    amazing     0.737888
       love     0.725421
       life     0.720616
       must     0.693609
      liked     0.657721
 characters     0.639486
    recipes     0.639084
        man     0.634798
  different     0.628354
       best     0.628130

-Top 20 negative-
           Word  Coefficient
           same    -0.776762
           idea    -0.782394
          waste    -0.791908
          found    -0.798260
          money    -0.803482
         finish    -0.811159
           bang    -0.820938
          whole    -0.842325
  disappointing    -0.858530
        nothing    -0.889667
             no    -0.897068
            off    -0.903228
 disappointment    -0.904

In [19]:
#base line accuracy (predicting with majority class, in this case positive class)
text_fit(X, y, c, DummyClassifier(),0)

# features: 15728
# train records: 3039
# test records: 1013
Model Accuracy: 0.9032576505429417


In [7]:
# logistic regression model on TFIDF
tfidf = TfidfVectorizer(stop_words = 'english')
text_fit(X, y, tfidf, LogisticRegression())


# features: 14706
# train records: 2600
# test records: 867
Model Accuracy: 0.9377162629757786

-Top 20 positive-
       Word  Coefficient
       read     2.097192
      great     1.848044
       love     1.424872
      loved     1.424724
       life     1.185892
  wonderful     1.075165
  recommend     1.004986
  excellent     0.963079
    amazing     0.952226
      story     0.951929
    enjoyed     0.930663
 characters     0.889129
      hotel     0.887167
         ve     0.875679
       king     0.748007
    recipes     0.726361
    history     0.686462
     russia     0.677448
      novel     0.667179
  beautiful     0.648612

-Top 20 negative-
          Word  Coefficient
           way    -0.992531
         trump    -1.047309
         pages    -1.102759
  disappointed    -1.107944
        wanted    -1.113789
           don    -1.136240
 disappointing    -1.177313
           fan    -1.183580
          didn    -1.248012
          star    -1.321241
          like    -1.461316
      

In [8]:
# logistic regression model on TFIDF + ngram
tfidf_n = TfidfVectorizer(ngram_range=(1,2),stop_words = 'english')
text_fit(X, y, tfidf_n, LogisticRegression())

# features: 111977
# train records: 2600
# test records: 867
Model Accuracy: 0.9377162629757786

-Top 20 positive-
       Word  Coefficient
       read     1.509999
      great     1.405842
       love     1.044878
      loved     1.034025
       life     0.865979
  wonderful     0.774926
      story     0.757285
  excellent     0.701451
  recommend     0.679522
      hotel     0.656050
 characters     0.648750
    amazing     0.639284
    enjoyed     0.607832
         ve     0.587655
 great book     0.567872
       king     0.550196
    history     0.530517
     russia     0.502507
      novel     0.488299
    recipes     0.482611

-Top 20 negative-
          Word  Coefficient
     did enjoy    -0.790974
          didn    -0.808642
        wanted    -0.813585
           don    -0.845786
   waste money    -0.846893
 disappointing    -0.848222
           fan    -0.879438
    waste time    -0.894877
     didn like    -0.896350
          star    -0.926436
   predictable    -0.994868
     

In [20]:
frames = [X, y]
df2 = pd.concat(frames, axis=1)
df2.head(5)

,review_title_text,review_score
0,Russian aristocracy following the Russian Revo...,1
1,Knowing nothing ahead of time just walk into t...,1
2,"One of King's finest, most frightening, & earl...",1
3,Not my favorite This book is t what I expecte...,1
4,"AMAZING, BOBBY As someone who hates reading a...",1


In [21]:
#split into training-validation for NB
ns_train, ns_test = train_test_split(df2, test_size=0.3)
ns_train.head(5)
ns_test.head(5)

#stm_train, stm_test = train_test_split(data_nostop_stemmed[['Corpus', 'Rating']], test_size=0.33)
#raw_train, raw_test = train_test_split(data_raw[['Corpus', 'Rating']], test_size=0.33)

,review_title_text,review_score
2442,Superb! Th Count has innumerable talents on d...,1
4058,A page turner ... This book will keep you in ...,1
2458,Baid and Switch I hate it when writers bait y...,0
1093,"Thoughtful , beautiful story. I wasn't sure I...",1
3682,Eye Opening A lot of valuable health informat...,1


In [22]:
#Naive Bayes
vectorizer = TfidfVectorizer()

ns_train_matrix = vectorizer.fit_transform(ns_train.review_title_text)
ns_train_label = ns_train.review_score
ns_test_matrix = vectorizer.transform(ns_test.review_title_text)
ns_test_label = ns_test.review_score

clf = MultinomialNB()
clf.fit(ns_train_matrix, ns_train_label)

ns_pred = clf.predict(ns_test_matrix)
ns_acc = accuracy_score(ns_test_label, ns_pred)
ns_f1 = f1_score(ns_test_label, ns_pred)

ns_prec = precision_score(ns_test_label, ns_pred)
ns_rec = recall_score(ns_test_label, ns_pred)

In [23]:
print('Accuracy: {0:.2f}%'.format(ns_acc*100))
print('Precision Score: {0:.2f}%'.format(ns_prec*100))
print('Recall Score: {0:.2f}%'.format(ns_rec*100))
print('F1 Score: {0:.4f}'.format(ns_f1))

Accuracy: 94.24%
Precision Score: 94.24%
Recall Score: 100.00%
F1 Score: 0.9704
